# IMPORTING NECESSARY LIBRARIES

Create a virtual environment and activate it (using 'conda activate _envname_' command ). Install the latest version of Python, latest version of qiskit (qiskit 1.0.2), networkx library, other libraries for ML and data visualisation like sklearn, matplotlib, numpy, in this virtual environment. 

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram, circuit_drawer, plot_state_qsphere
from qiskit_aer import AerSimulator
from qiskit_aer.noise import pauli_error, NoiseModel, depolarizing_error
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector, Kraus, SuperOp
from qiskit_ibm_runtime import SamplerV2

In [ ]:
import networkx as nx

In [ ]:
from qiskit.circuit import ParameterVector

The function for QAOA circuit for a MaxCut graph (to be generated later) is defined. It takes initial parameters, number of nodes of graph and the Graph as input. The initial parameters are random and includes both beta and gamma parameter values. Let d be the depth of QAOA, then len(initial parameters) = 2*d.

In [ ]:
def create_qaoa_circuit(params, n, G):
    p = len(params) // 2
    beta = params[:p]
    gamma = params[p:]
    
    qc = QuantumCircuit(n, n)
    
    # Initial layer of Hadamard gates
    for qubit in range(n):
        qc.h(qubit)
    
    for i in range(p):
        for edge in G.edges():
            qc.rzz(2 * gamma[i], edge[0], edge[1])
        for qubit in range(n):
            qc.rx(2 * beta[i], qubit)
    
    qc.measure(range(n), range(n))
    return qc

Expectation value for every instance of QAOA is calculated for a fixed d.

In [ ]:
# Function to compute the expectation value
def get_expectation_value(counts):
    value = 0
    for bitstring, count in counts.items():
        bit_val = np.array([1 if bit == '1' else -1 for bit in bitstring])
        value += sum(bit_val) * count
    return value / sum(counts.values())

Contains the updated code, compatible with Qiskit 1.0.2. No other code snippets available online are compatible with the recent version of qiskit.

In [ ]:
# Function to be minimized by the classical optimizer
def objective_function(params, n_nodes, graph):
    qc = create_qaoa_circuit(params, n_nodes, graph)
    backend = AerSimulator()
    result = backend.run(qc, shots=1024).result()
    counts = result.get_counts()
    return get_expectation_value(counts)

Synthetic data for one MAxcut Graph is generated

In [ ]:
from scipy.optimize import minimize
# Generate synthetic data for training
def generate_data(num_graphs, num_samples, n_nodes):
    X = []
    y = []
    depth =6

    for ___ in range (num_graphs):
        # Create random graph
        graph = nx.erdos_renyi_graph(n_nodes, 0.5)
        for _ in range(num_samples):
            # Random initial parameters
            init_params = np.random.rand(2 * depth) * np.pi
            
            # Optimize parameters using L-BFGS-B
            result = minimize(objective_function, init_params, args=(n_nodes, graph), method="L-BFGS-B")
            
            # Store the optimal parameters and the expectation value
            X.append(result.x)
            y.append(-result.fun)
    
    return np.array(X), np.array(y)


1 random MaxCut Graph with 8 nodes and node probability=0.5 is generated and 100 QAOA instances for the same are obtained, with expectation values of corresponding QAOA instance. Depth is set to 6.

In [ ]:
n_nodes = 8
num_samples = 6
num_graphs = 330
# Generate and split data
import time
#start = time.time()
X, y = generate_data(num_graphs, num_samples, n_nodes)
#end = time.time()

In [ ]:
# Print the collected data
print("Collected optimal parameters and expectation values:")
print(X)
print(y)

In [ ]:
X

In [ ]:
X.shape

In [ ]:
y

In [ ]:
y.shape